In [0]:
import numpy as np
import os
import tempfile
import keras
from keras import backend as K
from keras import layers
from keras.datasets import mnist
import tensorflow as tf

In [0]:
def cnn_layers(inputs):
    a = layers.Conv2D(32, (3, 3),
                      activation='relu', padding='valid')(inputs)
    a = layers.MaxPooling2D(pool_size=(2, 2))(a)
    a = layers.Conv2D(64, (3, 3), activation='relu')(a)
    a = layers.MaxPooling2D(pool_size=(2, 2))(a)
    a = layers.Flatten()(a)
    a = layers.Dense(512, activation='relu')(a)
    a = layers.Dropout(0.5)(a)
    preds = layers.Dense(num_classes,
                               activation='softmax',
                               name='a_train_out')(a)
    return preds

In [0]:
batch_size = 512
buffer_size = 10000
steps_per_epoch = int(np.ceil(60000 / float(batch_size)))  
epochs = 5
num_classes = 10

In [0]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.astype(np.float32) / 255
x_train = np.expand_dims(x_train, -1)
y_train = tf.one_hot(y_train, 10)

In [0]:
# Create the dataset and its associated iterator.
data = tf.data.Dataset.from_tensor_slices((x_train, y_train))
data = data.repeat()
data = data.shuffle(10000)
data = data.batch(512)
iterator = iter(data)

In [7]:
# Model creation using tensors from the get_next() graph node.
inputs, targets = iterator.get_next()
model_input = layers.Input(tensor=inputs)
model_output = cnn_layers(model_input)
model = keras.models.Model(inputs=model_input, outputs=model_output)
model.compile(optimizer=keras.optimizers.RMSprop(lr=2e-3, decay=1e-5),loss='categorical_crossentropy',metrics=['accuracy'],target_tensors=[targets])
model.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (512, 28, 28, 1)          0         
_________________________________________________________________
conv2d_3 (Conv2D)            (512, 26, 26, 32)         320       
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (512, 13, 13, 32)         0         
_________________________________________________________________
conv2d_4 (Conv2D)            (512, 11, 11, 64)         18496     
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (512, 5, 5, 64)           0         
_________________________________________________________________
flatten_2 (Flatten)          (512, 1600)               0         
_________________________________________________________________
dense_2 (Dense)              (512, 512)                8197

In [8]:
model.fit(epochs=5,steps_per_epoch=int(np.ceil(60000 / float(batch_size))))

Epoch 1/5
118/118 [==============================] - 43s 360ms/step - loss: 0.2006 - accuracy: 0.9412
Epoch 2/5
118/118 [==============================] - 42s 360ms/step - loss: 0.0371 - accuracy: 0.9959
Epoch 3/5
118/118 [==============================] - 43s 360ms/step - loss: 0.0146 - accuracy: 0.9979
Epoch 4/5
118/118 [==============================] - 42s 360ms/step - loss: 0.0175 - accuracy: 0.9978
Epoch 5/5
118/118 [==============================] - 47s 401ms/step - loss: 0.0089 - accuracy: 0.9984


In [0]:
# Save the model weights.
weight_path = os.path.join(tempfile.gettempdir(), 'saved_wt.h5')
model.save_weights(weight_path)

# Clean up the TF session.
K.clear_session()



In [10]:
# Second session to test loading trained model without tensors.
x_test = x_test.astype(np.float32)
x_test = np.expand_dims(x_test, -1)

x_test_inp = layers.Input(shape=x_test.shape[1:])
test_out = cnn_layers(x_test_inp)
test_model = keras.models.Model(inputs=x_test_inp, outputs=test_out)

test_model.load_weights(weight_path)
test_model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])
test_model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 28, 28, 1)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 1600)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 512)               8197

In [11]:
loss, acc = test_model.evaluate(x_test, y_test, 10)
print('\nTest accuracy: {0}'.format(acc))

10000/10000 [==============================] - 4s 381us/step

Test accuracy: 0.9487000107765198
